In [1]:
from vector_db.weaviate_db import WeaviateDB
from sentence_transformers import SentenceTransformer
from article_db.pandas_article_db import PandasArticleDB
from transformers import pipeline
from answer_extractor import AnswerExtractor

In [6]:
API_KEY = 'S0KbfM4Wx6g48PPzeZgKtTpQhj68fklwM9NH'
URL = 'https://rf-codes-na75gd7p.weaviate.network'
qa_model_name = 'AlexKay/xlm-roberta-large-qa-multilingual-finedtuned-ru'
st = SentenceTransformer('DeepPavlov/rubert-base-cased-sentence')
# articles = PandasArticleDB('/Users/mikhail_ovakimyan/Documents/Универ/Курсач по ТРПС/ai_legal_consultant/answer_extraction/db/labor_code.csv')
articles = PandasArticleDB('/Users/mikhail_ovakimyan/Documents/Универ/Курсач по ТРПС/ai_legal_consultant/answer_extraction/labor_code.csv')
qa_model = pipeline(model=qa_model_name, task='question-answering')
vector_db = WeaviateDB(URL, API_KEY)
ae = AnswerExtractor(vector_db, articles, qa_model, st)

No sentence-transformers model found with name /Users/mikhail_ovakimyan/.cache/torch/sentence_transformers/DeepPavlov_rubert-base-cased-sentence. Creating a new one with MEAN pooling.


In [18]:
ae.find('Работник имеет право на:')

{'part': 'Часть третья',
 'section': 'Раздел III',
 'chapter': 'Глава 12',
 'article': 'Статья 74',
 'article_text': 'При этом работнику предоставляются соответствующие гарантии и компенсации',
 'start': 8,
 'end': 73}

In [8]:
query = 'Богдана любят?'
enc_query = st.encode([query])
response = vector_db.find(enc_query)
contexts = [item['article_text'] for item in response]
contexts
# response

['Выполнение работником трудовой функции дистанционно не может являться основанием для снижения ему заработной платы. ',
 'При приеме на работу на срок до двух месяцев испытание работникам не устанавливается. ',
 'Продолжительность еженедельного непрерывного отдыха не может быть менее 42 часов, за исключением случаев, предусмотренных настоящим Кодексом. ',
 'По соглашению сторон трудовой договор между работником и работодателем - физическим лицом, не являющимся индивидуальным предпринимателем, может заключаться как на неопределенный, так и на определенный срок. ',
 'Особенности регулирования труда - нормы, частично ограничивающие применение общих правил по тем же вопросам либо предусматривающие для отдельных категорий работников дополнительные правила. ']

In [10]:
ids = [item['_additional']['id'] for item in response]
ids

['d00f22eb-5d96-4f0d-8659-52b1e263d0e6',
 '5fc19889-494a-4105-9b65-e4b057ce41ca',
 'fc76ffd5-9821-48c5-ba05-633478a6f6f6',
 '3d200246-34ec-4d8d-b31b-2fa445ffa88c',
 '4561b360-9d67-43a9-aa3e-e01f48a3265a']

In [15]:
results = []
for ctx_id, ctx in zip(ids, contexts):
    results.append((ctx_id, qa_model(question=query, context=ctx)))
results

[('d00f22eb-5d96-4f0d-8659-52b1e263d0e6',
  {'score': 0.04479764401912689,
   'start': 51,
   'end': 115,
   'answer': ' не может являться основанием для снижения ему заработной платы.'}),
 ('5fc19889-494a-4105-9b65-e4b057ce41ca',
  {'score': 0.37822532653808594,
   'start': 44,
   'end': 85,
   'answer': ' испытание работникам не устанавливается.'}),
 ('fc76ffd5-9821-48c5-ba05-633478a6f6f6',
  {'score': 0.020795678719878197,
   'start': 131,
   'end': 141,
   'answer': ' Кодексом.'}),
 ('3d200246-34ec-4d8d-b31b-2fa445ffa88c',
  {'score': 0.03117412142455578,
   'start': 43,
   'end': 70,
   'answer': ' работником и работодателем'}),
 ('4561b360-9d67-43a9-aa3e-e01f48a3265a',
  {'score': 0.08553966134786606, 'start': 33, 'end': 40, 'answer': ' нормы,'})]

In [16]:
final_result = max(results, key=lambda x: x[1]['score'])
final_result

('5fc19889-494a-4105-9b65-e4b057ce41ca',
 {'score': 0.37822532653808594,
  'start': 44,
  'end': 85,
  'answer': ' испытание работникам не устанавливается.'})

In [18]:
article_table = articles.get_table('labor_code')

response = {
    'part': article_table.loc[final_result[0], 'part'],
    'section': article_table.loc[final_result[0], 'section'],
    'chapter': article_table.loc[final_result[0], 'chapter'],
    'article': article_table.loc[final_result[0], 'article'],
    'article_text': article_table.loc[final_result[0], 'text'],
    'start': final_result[1]['start'],
    'end': final_result[1]['end'],
}
response

{'part': 'Часть четвертая',
 'section': 'Раздел XII',
 'chapter': 'Глава 45',
 'article': 'Статья 289',
 'article_text': 'При приеме на работу на срок до двух месяцев испытание работникам не устанавливается. ',
 'start': 44,
 'end': 85}

In [19]:
from answer_extractor import AnswerExtractor
ae = AnswerExtractor(vector_db, articles, qa_model, st)

In [20]:
ae.find('Богдана любят?')

KeyboardInterrupt: 

In [ ]:
ae = AnswerExtractor(vector_db, articles, qa_model, st)

In [10]:
query = 'Богдана любят?'
enc_query = st.encode([query]).tolist()
vector_db.index.query(enc_query)

[[-0.46361932158470154,
  -0.07614497095346451,
  -0.6344675421714783,
  -0.4401133060455322,
  -0.14963440597057343,
  -0.43319445848464966,
  0.4861203134059906,
  -0.717239499092102,
  0.20983067154884338,
  -0.5812354683876038,
  -0.7773714065551758,
  -0.11170829832553864,
  -0.6326481699943542,
  0.029700642451643944,
  -0.15123267471790314,
  -0.6284085512161255,
  0.5665010213851929,
  -1.0494277477264404,
  -1.009271502494812,
  -0.7269430160522461,
  0.5466044545173645,
  -0.29068103432655334,
  -0.6334624290466309,
  0.27932074666023254,
  0.6156684756278992,
  -0.5218475461006165,
  -0.17102846503257751,
  0.05698235705494881,
  0.9377368688583374,
  0.7783754467964172,
  -0.06519556045532227,
  0.09144558012485504,
  -0.820419192314148,
  -0.2933093011379242,
  -0.889613926410675,
  -0.20492962002754211,
  0.15596488118171692,
  1.8180011510849,
  -0.003766955342143774,
  -0.12189082056283951,
  -0.11315581947565079,
  0.4375092387199402,
  -0.8736101388931274,
  0.3110261

In [5]:
ae = AnswerExtractor(vector_db, articles, qa_model, st)

In [6]:
ae.find(['Богдана любят?'])

MaxRetryError: HTTPSConnectionPool(host='gcp-starter-unknown.svc.rf-code-pavlov.pinecone.io', port=443): Max retries exceeded with url: /query (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x172589b50>: Failed to resolve 'gcp-starter-unknown.svc.rf-code-pavlov.pinecone.io' ([Errno 8] nodename nor servname provided, or not known)"))

In [ ]:

query = st.encode(['Любят ли в Госдуме Богдана?']).tolist()

In [ ]:
vector_db.index = pinecone.Index('rf-code-pavlov')
vector_db.index.query(query, top_k=1)

In [ ]:
ds = Dataset.from_pandas(labor_codes)
ds = ds.map(lambda x: {
    'encoding': st.encode(x['text']).tolist()
}, batched=True, batch_size=32)

In [ ]:
ds = Dataset.from_pandas(labor_codes)
ds = ds.map(lambda x: {
    'encoding': st.encode(x['text']).tolist()
}, batched=True, batch_size=32)

In [ ]:

query = st.encode(['Любят ли в Госдуме Богдана?']).tolist()

In [ ]:
vector_db.index = pinecone.Index('rf-code-pavlov')
vector_db.index.query(query, top_k=1)

In [ ]:
ds = Dataset.from_pandas(labor_codes)
ds = ds.map(lambda x: {
    'encoding': st.encode(x['text']).tolist()
}, batched=True, batch_size=32)

In [ ]:
ds = Dataset.from_pandas(labor_codes)
ds = ds.map(lambda x: {
    'encoding': st.encode(x['text']).tolist()
}, batched=True, batch_size=32)

In [ ]:

query = st.encode(['Любят ли в Госдуме Богдана?']).tolist()

In [ ]:
vector_db.index = pinecone.Index('rf-code-pavlov')
vector_db.index.query(query, top_k=1)

In [ ]:
ds = Dataset.from_pandas(labor_codes)
ds = ds.map(lambda x: {
    'encoding': st.encode(x['text']).tolist()
}, batched=True, batch_size=32)

In [ ]:
ds = Dataset.from_pandas(labor_codes)
ds = ds.map(lambda x: {
    'encoding': st.encode(x['text']).tolist()
}, batched=True, batch_size=32)

In [ ]:
vector_db.index = pinecone.Index('rf-code-pavlov')
vector_db.index.query(query, top_k=1)

In [ ]:

query = st.encode(['Любят ли в Госдуме Богдана?']).tolist()

In [ ]:
vector_db.index = pinecone.Index('rf-code-pavlov')
vector_db.index.query(query, top_k=1)

In [ ]:
ds = Dataset.from_pandas(labor_codes)
ds = ds.map(lambda x: {
    'encoding': st.encode(x['text']).tolist()
}, batched=True, batch_size=32)

In [ ]:
ds = Dataset.from_pandas(labor_codes)
ds = ds.map(lambda x: {
    'encoding': st.encode(x['text']).tolist()
}, batched=True, batch_size=32)

In [ ]:
vector_db.index = pinecone.Index('rf-code-pavlov')
vector_db.index.query(query, top_k=1)

In [ ]:

query = st.encode(['Любят ли в Госдуме Богдана?']).tolist()

In [ ]:
vector_db.index = pinecone.Index('rf-code-pavlov')
vector_db.index.query(query, top_k=1)

In [ ]:
ds = Dataset.from_pandas(labor_codes)
ds = ds.map(lambda x: {
    'encoding': st.encode(x['text']).tolist()
}, batched=True, batch_size=32)

In [ ]:
ds = Dataset.from_pandas(labor_codes)
ds = ds.map(lambda x: {
    'encoding': st.encode(x['text']).tolist()
}, batched=True, batch_size=32)

In [ ]:
vector_db.index = pinecone.Index('rf-code-pavlov')
vector_db.index.query(query, top_k=1)

In [ ]:
vector_db.index = pinecone.Index('rf-code-pavlov')
vector_db.index.query(query, top_k=1)

In [ ]:
vector_db.index = pinecone.Index('rf-code-pavlov')
vector_db.index.query(query, top_k=1)

In [ ]:
ds = Dataset.from_pandas(labor_codes)
ds = ds.map(lambda x: {
    'encoding': st.encode(x['text']).tolist()
}, batched=True, batch_size=32)

In [ ]:
ds = Dataset.from_pandas(labor_codes)
ds = ds.map(lambda x: {
    'encoding': st.encode(x['text']).tolist()
}, batched=True, batch_size=32)

In [ ]:
vector_db.index = pinecone.Index('rf-code-pavlov')
vector_db.index.query(query, top_k=1)

In [ ]:
vector_db.index = pinecone.Index('rf-code-pavlov')
vector_db.index.query(query, top_k=1)

In [ ]:
ds = Dataset.from_pandas(labor_codes)
ds = ds.map(lambda x: {
    'encoding': st.encode(x['text']).tolist()
}, batched=True, batch_size=32)

In [ ]:
ds = Dataset.from_pandas(labor_codes)
ds = ds.map(lambda x: {
    'encoding': st.encode(x['text']).tolist()
}, batched=True, batch_size=32)

In [ ]:
vector_db.index = pinecone.Index('rf-code-pavlov')
vector_db.index.query(query, top_k=1)

In [ ]:
vector_db.index = pinecone.Index('rf-code-pavlov')
vector_db.index.query(query, top_k=1)

In [ ]:
ds = Dataset.from_pandas(labor_codes)
ds = ds.map(lambda x: {
    'encoding': st.encode(x['text']).tolist()
}, batched=True, batch_size=32)

In [ ]:
ds = Dataset.from_pandas(labor_codes)
ds = ds.map(lambda x: {
    'encoding': st.encode(x['text']).tolist()
}, batched=True, batch_size=32)

In [ ]:
vector_db.index = pinecone.Index('rf-code-pavlov')
vector_db.index.query(query, top_k=1)

In [ ]:
vector_db.index = pinecone.Index('rf-code-pavlov')
vector_db.index.query(query, top_k=1)

In [ ]:
ds = Dataset.from_pandas(labor_codes)
ds = ds.map(lambda x: {
    'encoding': st.encode(x['text']).tolist()
}, batched=True, batch_size=32)

In [ ]:
ds = Dataset.from_pandas(labor_codes)
ds = ds.map(lambda x: {
    'encoding': st.encode(x['text']).tolist()
}, batched=True, batch_size=32)

In [ ]:
vector_db.index = pinecone.Index('rf-code-pavlov')
vector_db.index.query(query, top_k=1)

In [ ]:
vector_db.index = pinecone.Index('rf-code-pavlov')
vector_db.index.query(query, top_k=1)

In [ ]:
ds = Dataset.from_pandas(labor_codes)
ds = ds.map(lambda x: {
    'encoding': st.encode(x['text']).tolist()
}, batched=True, batch_size=32)

In [ ]:
ds = Dataset.from_pandas(labor_codes)
ds = ds.map(lambda x: {
    'encoding': st.encode(x['text']).tolist()
}, batched=True, batch_size=32)

In [ ]:
vector_db.index = pinecone.Index('rf-code-pavlov')
vector_db.index.query(query, top_k=1)

In [ ]:
vector_db.index = pinecone.Index('rf-code-pavlov')
vector_db.index.query(query, top_k=1)

In [ ]:
ds = Dataset.from_pandas(labor_codes)
ds = ds.map(lambda x: {
    'encoding': st.encode(x['text']).tolist()
}, batched=True, batch_size=32)

In [ ]:
ds = Dataset.from_pandas(labor_codes)
ds = ds.map(lambda x: {
    'encoding': st.encode(x['text']).tolist()
}, batched=True, batch_size=32)

In [ ]:
vector_db.index = pinecone.Index('rf-code-pavlov')
vector_db.index.query(query, top_k=1)

In [ ]:
ds = Dataset.from_pandas(labor_codes)
ds = ds.map(lambda x: {
    'encoding': st.encode(x['text']).tolist()
}, batched=True, batch_size=32)

In [ ]:
ds = Dataset.from_pandas(labor_codes)
ds = ds.map(lambda x: {
    'encoding': st.encode(x['text']).tolist()
}, batched=True, batch_size=32)

In [ ]:
vector_db.index = pinecone.Index('rf-code-pavlov')
vector_db.index.query(query, top_k=1)

In [ ]:
ds = Dataset.from_pandas(labor_codes)
ds = ds.map(lambda x: {
    'encoding': st.encode(x['text']).tolist()
}, batched=True, batch_size=32)

In [ ]:
ds = Dataset.from_pandas(labor_codes)
ds = ds.map(lambda x: {
    'encoding': st.encode(x['text']).tolist()
}, batched=True, batch_size=32)

In [ ]:
vector_db.index = pinecone.Index('rf-code-pavlov')
vector_db.index.query(query, top_k=1)

In [ ]:
ds = Dataset.from_pandas(labor_codes)
ds = ds.map(lambda x: {
    'encoding': st.encode(x['text']).tolist()
}, batched=True, batch_size=32)

In [ ]:
ds = Dataset.from_pandas(labor_codes)
ds = ds.map(lambda x: {
    'encoding': st.encode(x['text']).tolist()
}, batched=True, batch_size=32)

In [ ]:
vector_db.index = pinecone.Index('rf-code-pavlov')
vector_db.index.query(query, top_k=1)

In [ ]:
ds = Dataset.from_pandas(labor_codes)
ds = ds.map(lambda x: {
    'encoding': st.encode(x['text']).tolist()
}, batched=True, batch_size=32)

In [ ]:
ds = Dataset.from_pandas(labor_codes)
ds = ds.map(lambda x: {
    'encoding': st.encode(x['text']).tolist()
}, batched=True, batch_size=32)

In [ ]:
vector_db.index = pinecone.Index('rf-code-pavlov')
vector_db.index.query(query, top_k=1)

In [ ]:
ds = Dataset.from_pandas(labor_codes)
ds = ds.map(lambda x: {
    'encoding': st.encode(x['text']).tolist()
}, batched=True, batch_size=32)

In [ ]:
ds = Dataset.from_pandas(labor_codes)
ds = ds.map(lambda x: {
    'encoding': st.encode(x['text']).tolist()
}, batched=True, batch_size=32)

In [ ]:
vector_db.index = pinecone.Index('rf-code-pavlov')
vector_db.index.query(query, top_k=1)

In [ ]:
ds = Dataset.from_pandas(labor_codes)
ds = ds.map(lambda x: {
    'encoding': st.encode(x['text']).tolist()
}, batched=True, batch_size=32)

In [ ]:
ds = Dataset.from_pandas(labor_codes)
ds = ds.map(lambda x: {
    'encoding': st.encode(x['text']).tolist()
}, batched=True, batch_size=32)

In [ ]:
vector_db.index = pinecone.Index('rf-code-pavlov')
vector_db.index.query(query, top_k=1)

In [ ]:
ds = Dataset.from_pandas(labor_codes)
ds = ds.map(lambda x: {
    'encoding': st.encode(x['text']).tolist()
}, batched=True, batch_size=32)

In [ ]:
ds = Dataset.from_pandas(labor_codes)
ds = ds.map(lambda x: {
    'encoding': st.encode(x['text']).tolist()
}, batched=True, batch_size=32)

In [9]:
vector_db.index = pinecone.Index('rf-code-pavlov')
vector_db.index.query(query, top_k=1)

PineconeProtocolError: Failed to connect; did you specify the correct index name?

In [ ]:
ds = Dataset.from_pandas(labor_codes)
ds = ds.map(lambda x: {
    'encoding': st.encode(x['text']).tolist()
}, batched=True, batch_size=32)

In [ ]:
ds = Dataset.from_pandas(labor_codes)
ds = ds.map(lambda x: {
    'encoding': st.encode(x['text']).tolist()
}, batched=True, batch_size=32)

In [ ]:
ds = Dataset.from_pandas(labor_codes)
ds = ds.map(lambda x: {
    'encoding': st.encode(x['text']).tolist()
}, batched=True, batch_size=32)

In [8]:
ds = Dataset.from_pandas(labor_codes)
ds = ds.map(lambda x: {
    'encoding': st.encode(x['text']).tolist()
}, batched=True, batch_size=32)

Map:   0%|          | 0/578 [00:00<?, ? examples/s]

In [1]:
import http.server